In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization, AveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Lambda
from keras import backend as K


# dimensions of our images.
img_width, img_height = 300, 300

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 3000
nb_validation_samples = 300
epochs = 5
batch_size = 50

Using TensorFlow backend.


In [2]:
input_shape = (img_height, img_width, 3)

## Adding more convolutions + dropout

In [10]:
def ConvBlock(model, filters):            
    model.add(Conv2D(filters, (3, 3), activation='relu'))        
    model.add(MaxPooling2D((2,2)))    

In [11]:
model = Sequential()
model.add(Lambda(lambda x: x, input_shape=input_shape))
ConvBlock(model, 20)
ConvBlock(model, 40)
model.add(Dropout(0.1))
ConvBlock(model, 60)
ConvBlock(model, 80)
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='softmax'))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 300, 300, 3)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 298, 298, 20)      560       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 20)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 40)      7240      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 40)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 73, 73, 40)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 60)        21660     
__________

In [13]:
model.compile(loss='categorical_crossentropy',optimizer='adadelta',metrics=['accuracy'])

In [14]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical')

Found 3000 images belonging to 2 classes.
Found 300 images belonging to 2 classes.


In [15]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps= 2)

Epoch 1/5
60/60 [==============================] - 52s - loss: 0.6993 - acc: 0.5370 - val_loss: 0.9752 - val_acc: 0.5200